# UCID Basics: Introduction to Urban Context Identifiers

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ucid-foundation/ucid/blob/main/notebooks/00_ucid_basics.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ucid-foundation/ucid/main?labpath=notebooks%2F00_ucid_basics.ipynb)

---

## Overview

This notebook provides a comprehensive introduction to the **Urban Context Identifier (UCID)** system. You will learn:

1. What UCID is and why it matters
2. The UCID format specification (UCID-V1)
3. How to create, parse, and validate UCIDs
4. Working with UCID components
5. Batch processing and performance considerations

### Prerequisites

- Python 3.11 or later
- Basic understanding of geospatial concepts
- Familiarity with pandas DataFrames

### Learning Objectives

By the end of this notebook, you will be able to:

- Create UCIDs from coordinates and timestamps
- Parse and validate existing UCID strings
- Extract individual components from UCIDs
- Process large batches of UCIDs efficiently

---

## 1. Environment Setup

First, let's install and import the required dependencies.

In [ ]:
# Install UCID if not already installed
%pip install -q ucid

In [ ]:
# Standard library imports

# Third-party imports
import numpy as np
import pandas as pd

# UCID imports
import ucid
from ucid import create_ucid, is_valid_ucid, parse_ucid
from ucid.core.errors import UCIDParseError, UCIDValidationError

print(f"UCID version: {ucid.__version__}")
print(f"Python version: {pd.show_versions}")

---

## 2. Understanding the UCID Format

### 2.1 What is a UCID?

A **UCID (Urban Context Identifier)** is a standardized string that uniquely identifies a location in space and time, along with its urban context score. The format follows this structure:

```
UCID:V1:{CITY}:{H3_INDEX}:{TIMESTAMP}:{CONTEXT}:{SCORE}:{GRADE}:{CONFIDENCE}
```

### 2.2 UCID Components

| Component | Description | Example |
|-----------|-------------|----------|
| Prefix | Always "UCID" | UCID |
| Version | Format version | V1 |
| City | UN/LOCODE city code | IST |
| H3 Index | H3 hexagonal cell ID | 8a1fb46622dffff |
| Timestamp | ISO week + hour | 2026W02T14 |
| Context | Scoring context | 15MIN |
| Score | Numeric score (0-100) | 72 |
| Grade | Letter grade (A-F) | B |
| Confidence | Confidence level | 95 |

### 2.3 Example UCID

```
UCID:V1:IST:8a1fb46622dffff:2026W02T14:15MIN:72:B:95
```

This UCID represents:
- A location in Istanbul (IST)
- At H3 resolution 10, cell 8a1fb46622dffff
- During week 2 of 2026, at 14:00
- Scored using the 15-minute city context
- With a score of 72 (grade B)
- At 95% confidence

---

## 3. Creating UCIDs

### 3.1 Basic UCID Creation

The simplest way to create a UCID is using the `create_ucid()` function with coordinates.

In [ ]:
# Create a basic UCID for Istanbul
ucid_string = create_ucid(
    city="IST",
    lat=41.0082,
    lon=28.9784,
    timestamp="2026W02T14",
    context="15MIN",
)

print(f"Created UCID: {ucid_string}")
print(f"Length: {len(ucid_string)} characters")

In [ ]:
# Create UCIDs for different global cities
cities = [
    {"city": "NYC", "lat": 40.7128, "lon": -74.0060, "name": "New York"},
    {"city": "LON", "lat": 51.5074, "lon": -0.1278, "name": "London"},
    {"city": "TYO", "lat": 35.6762, "lon": 139.6503, "name": "Tokyo"},
    {"city": "PAR", "lat": 48.8566, "lon": 2.3522, "name": "Paris"},
    {"city": "SYD", "lat": -33.8688, "lon": 151.2093, "name": "Sydney"},
]

print("UCIDs for Major World Cities:")
print("=" * 80)

for city_data in cities:
    city_ucid = create_ucid(
        city=city_data["city"],
        lat=city_data["lat"],
        lon=city_data["lon"],
        timestamp="2026W02T14",
        context="15MIN",
    )
    print(f"{city_data['name']:12s}: {city_ucid}")

### 3.2 UCID Creation with Different Contexts

UCID supports multiple urban context types for different analytical perspectives.

In [ ]:
# Available contexts
contexts = ["15MIN", "TRANSIT", "CLIMATE", "VITALITY", "EQUITY", "WALK"]

print("Same Location with Different Contexts:")
print("=" * 80)

for ctx in contexts:
    ctx_ucid = create_ucid(
        city="IST",
        lat=41.0082,
        lon=28.9784,
        timestamp="2026W02T14",
        context=ctx,
    )
    print(f"{ctx:10s}: {ctx_ucid}")

### 3.3 UCID Creation with Custom Parameters

In [ ]:
# Create UCID with explicit H3 resolution
ucid_high_res = create_ucid(
    city="IST",
    lat=41.0082,
    lon=28.9784,
    timestamp="2026W02T14",
    context="15MIN",
    resolution=11,  # Higher resolution (smaller hexagons)
)

ucid_low_res = create_ucid(
    city="IST",
    lat=41.0082,
    lon=28.9784,
    timestamp="2026W02T14",
    context="15MIN",
    resolution=8,  # Lower resolution (larger hexagons)
)

print(f"High resolution (11): {ucid_high_res}")
print(f"Low resolution (8):  {ucid_low_res}")

---

## 4. Parsing UCIDs

### 4.1 Basic Parsing

In [ ]:
# Parse an existing UCID string
sample_ucid = "UCID:V1:IST:8a1fb46622dffff:2026W02T14:15MIN:72:B:95"

parsed = parse_ucid(sample_ucid)

print("Parsed UCID Components:")
print("=" * 40)
print(f"City:       {parsed.city}")
print(f"H3 Index:   {parsed.h3_index}")
print(f"Timestamp:  {parsed.timestamp}")
print(f"Context:    {parsed.context}")
print(f"Score:      {parsed.score}")
print(f"Grade:      {parsed.grade}")
print(f"Confidence: {parsed.confidence}")

In [ ]:
# Access geographic coordinates from parsed UCID
lat, lon = parsed.to_coordinates()
print(f"Coordinates: ({lat:.6f}, {lon:.6f})")

# Get the datetime from timestamp
dt = parsed.to_datetime()
print(f"Datetime: {dt}")

### 4.2 Validation

In [ ]:
# Validate UCID strings
test_ucids = [
    "UCID:V1:IST:8a1fb46622dffff:2026W02T14:15MIN:72:B:95",  # Valid
    "UCID:V2:IST:8a1fb46622dffff:2026W02T14:15MIN:72:B:95",  # Invalid version
    "INVALID_STRING",  # Invalid format
    "UCID:V1:XXX:8a1fb46622dffff:2026W02T14:15MIN:72:B:95",  # Invalid city
]

print("UCID Validation Results:")
print("=" * 80)

for test_ucid in test_ucids:
    is_valid = is_valid_ucid(test_ucid)
    status = "Valid" if is_valid else "Invalid"
    print(f"{status:8s} | {test_ucid[:50]}...")

### 4.3 Error Handling

In [ ]:
# Demonstrate error handling
invalid_ucids = [
    ("UCID:V2:IST:abc:2026W02T14:15MIN:72:B:95", "Invalid version"),
    ("NOT_A_UCID", "Invalid format"),
    ("UCID:V1:IST:8a1fb46622dffff:2026W02T14:15MIN:150:B:95", "Score out of range"),
]

for invalid_ucid, description in invalid_ucids:
    try:
        parse_ucid(invalid_ucid)
        print(f"Unexpectedly parsed: {description}")
    except (UCIDValidationError, UCIDParseError) as e:
        print(f"{description}: {type(e).__name__}")

---

## 5. Batch Processing

### 5.1 Creating Multiple UCIDs

In [ ]:
# Generate random locations within Istanbul
np.random.seed(42)
n_locations = 1000

lat_min, lat_max = 40.8, 41.2
lon_min, lon_max = 28.6, 29.4

lats = np.random.uniform(lat_min, lat_max, n_locations)
lons = np.random.uniform(lon_min, lon_max, n_locations)

# Create UCIDs in batch
ucids = []
for lat, lon in zip(lats, lons):
    ucid_str = create_ucid(
        city="IST",
        lat=lat,
        lon=lon,
        timestamp="2026W02T14",
        context="15MIN",
    )
    ucids.append(ucid_str)

print(f"Created {len(ucids)} UCIDs")
print("Sample UCIDs:")
for ucid_str in ucids[:5]:
    print(f"  {ucid_str}")

### 5.2 Working with DataFrames

In [ ]:
# Create a DataFrame with UCIDs
df = pd.DataFrame(
    {
        "ucid": ucids,
        "latitude": lats,
        "longitude": lons,
    }
)

# Parse UCIDs to extract components
df["score"] = df["ucid"].apply(lambda x: parse_ucid(x).score)
df["grade"] = df["ucid"].apply(lambda x: parse_ucid(x).grade)
df["h3_index"] = df["ucid"].apply(lambda x: parse_ucid(x).h3_index)

print("DataFrame with UCIDs:")
df.head(10)

In [ ]:
# Summary statistics
print("Score Distribution:")
print(df["score"].describe())

print("\nGrade Distribution:")
print(df["grade"].value_counts().sort_index())

---

## 6. Performance Benchmarks

### 6.1 Creation Performance

In [ ]:
import time

# Benchmark UCID creation
n_iterations = 10000

start_time = time.perf_counter()
for _ in range(n_iterations):
    _ = create_ucid(
        city="IST",
        lat=41.0082,
        lon=28.9784,
        timestamp="2026W02T14",
        context="15MIN",
    )
end_time = time.perf_counter()

elapsed = end_time - start_time
rate = n_iterations / elapsed

print("UCID Creation Performance:")
print(f"  Iterations: {n_iterations:,}")
print(f"  Total time: {elapsed:.3f} seconds")
print(f"  Rate: {rate:,.0f} UCIDs/second")

---

## 7. Summary

In this notebook, you learned:

1. **UCID Format**: The structure and components of Urban Context Identifiers
2. **Creation**: How to create UCIDs using `create_ucid()`
3. **Parsing**: How to parse and extract components with `parse_ucid()`
4. **Validation**: How to validate UCID strings with `is_valid_ucid()`
5. **Batch Processing**: Working with multiple UCIDs efficiently

### Next Steps

Continue with the following notebooks:

- **01_citygrid_scan_basics.ipynb**: Learn to scan entire cities with UCID grids
- **02_15min_city_isochrones.ipynb**: Explore 15-minute city analysis
- **03_transit_gtfs_quality.ipynb**: Analyze transit accessibility

---

## References

- [UCID Documentation](https://ucid.readthedocs.io/)
- [H3 Spatial Indexing](https://h3geo.org/)
- [UN/LOCODE](https://unece.org/trade/cefact/unlocode-code-list-country-and-territory)

---

*Copyright 2026 UCID Foundation. Licensed under EUPL-1.2.*